In [0]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [0]:
# 0. prepare data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

## scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

In [0]:
# 1. model
class LogisticRegression(nn.Module):

    def __init__(self, n_input_features):
        super().__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_predict = torch.sigmoid(self.linear(x))
        return y_predict

model = LogisticRegression(n_features)

In [0]:
# 2. loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [0]:
# 3. training loss
num_epochs = 100
for epoch in range(num_epochs):
    # forward and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)
    # backward
    loss.backward()
    # optim
    optimizer.step()
    # zero_gradient
    optimizer.zero_grad()

    if epoch % 10 == 1:
        print(f'epoch : {epoch}, loss = {loss.item():.4f}')
# test
with torch.no_grad() :
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round() # classification이니까 0.5이상은 1 나머지는 0
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

epoch : 1, loss = 0.0913
epoch : 11, loss = 0.0911
epoch : 21, loss = 0.0909
epoch : 31, loss = 0.0907
epoch : 41, loss = 0.0906
epoch : 51, loss = 0.0904
epoch : 61, loss = 0.0902
epoch : 71, loss = 0.0901
epoch : 81, loss = 0.0899
epoch : 91, loss = 0.0897
accuracy = 0.9650
